<a href="https://colab.research.google.com/github/AdityaKuranjekarMYCAPTAIN/Banking-System/blob/main/TextGenratorProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import dependencies
import numpy
import sys
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

In [3]:
# load data
# loading data and opening our input data in the form of txt form
# Project Gutenberg/berg is where the data can be jound (Just google it)
file = open("/Frankenstein.txt", encoding='latin-1').read()

Tokenization is the process of breaking a big piece of text, like a sentence or paragraph, into smaller parts called tokens. These tokens can be words, phrases, or even individual characters, depending on the purpose.

In [5]:
# tokenization
# WHat is tokanization? Process of breaking a stream of text into words, phrase or symbols or meaningful elements
# standardization
nltk.download('stopwords')
def tokenize_words(input):
  # Lowercase everything to standardize it
  input = input.lower()
  # Initiating tokenizer
  tokenizer = RegexpTokenizer(r'\w+')
  tokens = tokenizer.tokenize(input)
  # Filtering the stopwords using lambda
  filtered = filter(lambda token: token not in stopwords.words('english'), tokens)
  return "".join(filtered)

# Preprocessing the input data, make tokens
processed_inputs = tokenize_words(file)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Our Tokenization task is now completed

Nural network essentially wok with numbers not chaarcters.
So we have to convert char into numbers

In [6]:
# Char to numbers
# Convert characters in our input to numbers
# we'll sort the list of the set of all characters that appera in out i/p text and then enumerate fn
# to get numbers that represent the chaaracters
# we'll then create dictionaries that stores the key and value, or the characters and numbers that represent them
chars = sorted(list(set(processed_inputs)))
char_to_num = dict((c, i) for i, c in enumerate(chars))

In [7]:
# Check if the words to chars or chars to num is worked??
# just so that we get idea of weather our process of converting words into char has worked
# We print the length of out variables
input_len = len(processed_inputs)
vocab_len = len(chars)
print ("Total number of characters:", input_len)
print ("Total vocab:", vocab_len)

Total number of characters: 212998
Total vocab: 41


In [8]:
# Seq length
# we're defining how long we want individual sequence
# an individual sequence is an complete mapping of input characters as integers
seq_length = 100
x_data = []
y_data = []

In [9]:
# loop through the sequence
# here we are going through the entire lists of i/ps and converting the chars to numbres with a for loop
# this will create bunch of sequence where each sequence starrts with the next char in the i/p data
# beginning with the first character
for i in range(0, input_len - seq_length, 1):
  # define i/p and o/p sequence
  # i/p is current character plus the desired sequence length
  in_seq = processed_inputs[i:i + seq_length]
  # out sequence is the initial character plus total sequence length
  out_seq = processed_inputs[i + seq_length]
  # Converting the list of characters to integers based on previous values and appending the values to or list
  x_data.append([char_to_num[char] for char in in_seq])
  y_data.append(char_to_num[out_seq])

# Check to see how many total input seuence we have
n_patterns = len(x_data)
print ("Total Patterns:", n_patterns)

Total Patterns: 212898


In [10]:
# Convert input sequence to np array that our network can use
X = numpy.reshape(x_data, (n_patterns, seq_length, 1))
X = X/float(vocab_len)

In [11]:
# one-hot encoding our label data
y = to_categorical(y_data)

In [12]:
# Creating the model
# creating sequential model
# Dropout is used to prevent overfitting
# LSTM
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [13]:
# Compile
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [14]:
# saving weights
from tensorflow.keras.callbacks import ModelCheckpoint

# Saving weights with .keras format (recommended for TensorFlow >= 2.6)
filepath = "model_weights_saved.keras"  # Change extension to .keras
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
desired_callbacks = [checkpoint]

In [15]:
# fit model and let it train
model.fit(X, y, epochs=4, batch_size=256, callbacks=desired_callbacks)

Epoch 1/4
832/832 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - loss: 2.9742
Epoch 1: loss improved from inf to 2.93354, saving model to model_weights_saved.keras
832/832 ━━━━━━━━━━━━━━━━━━━━ 3377s 4s/step - loss: 2.9742
Epoch 2/4
832/832 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - loss: 2.9077
Epoch 2: loss improved from 2.93354 to 2.91030, saving model to model_weights_saved.keras
832/832 ━━━━━━━━━━━━━━━━━━━━ 3367s 4s/step - loss: 2.9077
Epoch 3/4
832/832 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - loss: 2.9043
Epoch 3: loss improved from 2.91030 to 2.90503, saving model to model_weights_saved.keras
832/832 ━━━━━━━━━━━━━━━━━━━━ 3379s 4s/step - loss: 2.9043
Epoch 4/4
832/832 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - loss: 2.8861
Epoch 4: loss improved from 2.90503 to 2.87842, saving model to model_weights_saved.keras
832/832 ━━━━━━━━━━━━━━━━━━━━ 3372s 4s/step - loss: 2.8861


In [16]:
# recompile model with saved weights
filename = "model_weights_saved.keras"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [17]:
# output of the model back into chatacter
num_to_char = dict((i, c) for i, c in enumerate(chars))

In [33]:
# random seed to help generate
start = numpy.random.randint(0, len(x_data) - 1)
pattern = x_data[start]
print("Random Seed:")
print("\"", ' '.join([num_to_char[value] for value in pattern]), "\" ")

Random Seed:
" e r e d c h a s e a w a y i d l e f e a r s a l o n e c o n s e c r a t e l i f e e n d e a v o u r s c o n t e n t m e n t o n e s e c r e t e l i z a b e t h d r e a d f u l o n e r e v e a l e d c " 


In [35]:
# generate the text
for i in range(1000):
  x = numpy.reshape(pattern, (1, len(pattern), 1))
  x = x / float(vocab_len)
  prediction = model.predict(x, verbose=0)
  index = numpy.argmax(prediction)
  result = num_to_char[index]
  seq_in = [num_to_char[value] for value in pattern]
  sys.stdout.write(result)
  pattern.append(index)
  pattern = pattern[1:len(pattern)]

eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee

RuntimeError: input_dataset: Attempting to capture an EagerTensor without building a function.